<a href="https://colab.research.google.com/github/Yulianagalvis/Se-ales-y-sistemas/blob/main/PARCIAL_2_Defiintivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PARCIAL 2 - SEÑALES Y SISTEMAS

# INTEGRANTES: CRISTIAN ARMANDO CHAMORRO MELO - YULIANA ALEXANDRA GALVIS CARDONA - JUAN MANUEL MEJÍA VASCO


1. Encuentre la expresión del espectro de Fourier (forma exponencial y trigonométrica) para la señal x(t) = |A sin(2πF₀t)|², con t ∈ [−1/(2F₀), 1/(2F₀)], con A, F₀ ∈ ℝ⁺.


2. Realice las simulaciones respectivas para graficar el espectro de Fourier del ejercicio 1 (magnitud y fase como diagrama de Bode en decibelios), y presente el error relativo y la senal reconstruida para N = {1, 2, . . . , 50}.

In [ ]:
%%writefile TFourier.py

import numpy as np
import matplotlib.pyplot as plt
import streamlit as st

st.title('Simulación de transformada de fourier de una señal')

# Parámetros de la señal
A = 8  # Amplitud fija
Fo = 1  # Frecuencia fundamental fija en Hz
Fs = 100 * Fo  # Frecuencia de muestreo
ti = -1 / 2*Fo  # Tiempo inicial
tf = 1 / 2*Fo   # Tiempo final
T = tf - ti   # Intervalo de interés
N = 50  # Número de armónicos fijos

wo = 2 * np.pi / T  # Frecuencia angular
tv = np.arange(ti, tf, 1 / Fs)  # Vector de tiempo

# Definir bases
phin = np.zeros((len(tv), 2 * N + 1), dtype=np.complex_)
for n in range(-N, N + 1, 1):
    phin[:, n + N] = np.exp(1j * n * wo * tv)  # Base de Fourier

# graficar bases##########
from ipywidgets import interact,IntSlider

def pltbase(n=1):
    plt.plot(tv,np.real(phin[:,int(n+N)]),label="Re{$\phi_n$(t)}")
    plt.xlabel("t[s]",fontsize = 14)
    plt.ylabel("$\phi_n$(t)",fontsize = 14)
    plt.plot(tv,np.imag(phin[:,int(n+N)]),label="Im{$\phi_n$(t)}")
    plt.grid()
    plt.legend()
    plt.show()
    return
interact(pltbase,n=(-N,N,1))

# Calcular el espectro de Fourier (según la señal estudiada)
cn = np.zeros(2 * N + 1, dtype=np.complex_)
nv = np.linspace(-N, N, 2 * N + 1)  # Vector de armónicos
cn[N] = A**2 / 2  # Nivel DC (C_0)
cn[N - 2] = -(A**2 / 4)  # Armónico -2
cn[N + 2] = -(A**2 / 4)  # Armónico +2

# Señal original
x = abs(A * np.sin(2 * np.pi * Fo * tv)) ** 2

# Potencia de la señal original
Px = 3 * (A**4) / 8

st.write("""
El objetivo de realizar las simulaciones para obtener el espectro de Fourier en sus formas exponencial y trigonométrica de la señal
$$ x(t) = |A \sin(2\pi F_0 t)|^2 $$
es entender cómo esta señal se descompone en componentes de frecuencia.
""")
# 1. Función para mostrar la señal original
def plot_original_signal():
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(tv, x, color='r', label='$x(t)$ (Señal original)')
    ax.set_title('Señal original $x(t)$', fontsize=16)
    ax.set_xlabel('t [s]')
    ax.set_ylabel('x(t)')
    ax.grid(True)
    ax.legend()

    st.pyplot(fig)

st.write("""
Las simulaciones ayudan a visualizar cómo la serie de Fourier aproxima una señal periódica. Se simulan los coeficientes de Fourier
$$ C_n $$ (en la forma exponencial) o
$$ a_n, b_n $$ (en la forma trigonométrica), que son los pesos de las componentes sinusoidales.
""")
# 2. Función para mostrar la señal con diferentes armónicos en el tiempo
def plot_harmonics_time(Na):
    ind = range(N - Na, N + Na + 1)

    # Señal filtrada por los armónicos seleccionados
    xe = phin[:, ind].dot(cn[ind])

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(tv, xe, color='b', label=f'Señal con {Na} armónicos')
    ax.set_title(f'Señal en el tiempo con {Na} armónicos', fontsize=16)
    ax.set_xlabel('t [s]')
    ax.set_ylabel('x(t)')
    ax.grid(True)
    ax.legend()

    st.pyplot(fig)

st.write("""
Al calcular y graficar el espectro de Fourier (magnitud y fase), podemos observar cuáles son las frecuencias dominantes de la señal y cómo está distribuida la energía en el dominio de la frecuencia.
""")
# 3. Función para graficar el espectro de Fourier
def plot_spectrum():
    fig, axs = plt.subplots(2, 2, figsize=(10, 8))

    # Parte real de C_n
    axs[0, 0].stem(nv, np.real(cn), 'r', use_line_collection=True)
    axs[0, 0].set_xlabel(r'$n \cdot w_o$ [rad/s]')
    axs[0, 0].set_ylabel(r'Re$\{C_n\}$')
    axs[0, 0].grid(True)

    # Parte imaginaria de C_n
    axs[0, 1].stem(nv, np.imag(cn), 'r', use_line_collection=True)
    axs[0, 1].set_xlabel(r'$n \cdot w_o$ [rad/s]')
    axs[0, 1].set_ylabel(r'Im$\{C_n\}$')
    axs[0, 1].grid(True)

    # Magnitud de C_n en dB
    axs[1, 0].stem(nv, 20 * np.log10(np.abs(cn)), 'r', use_line_collection=True)
    axs[1, 0].set_xlabel(r'$n \cdot w_o$ [rad/s]')
    axs[1, 0].set_ylabel(r'$|C_n|$ [dB]')
    axs[1, 0].grid(True)

    # Fase de C_n
    axs[1, 1].stem(nv, np.angle(cn), 'r', use_line_collection=True)
    axs[1, 1].set_xlabel(r'$n \cdot w_o$ [rad/s]')
    axs[1, 1].set_ylabel(r'$\angle C_n$ [rad]')
    axs[1, 1].grid(True)

    # Ajustar el layout de las subplots
    fig.tight_layout()

    # Mostrar la gráfica en Streamlit
    st.pyplot(fig)

st.write("""
## Reconstrucción de la señal y cálculo del error relativo:

Uno de los objetivos clave es ver cómo la serie de Fourier puede aproximar la señal original usando un número finito de términos (armónicos). A medida que se aumenta el número de armónicos, la aproximación de la señal mejora.

Para $$ N = 1, 2, \dots, 50 $$, la señal se reconstruye con diferentes cantidades de armónicos, y se muestra el error relativo entre la señal original y la señal reconstruida.
""")

# Bloque 6: Evaluación de la reconstrucción
st.write("""
Esto nos permite:
- Evaluar la calidad de la reconstrucción.
- Ver cuántos términos son suficientes para aproximar la señal con un error aceptable.
- Analizar cómo los primeros armónicos capturan las características principales de la señal y cómo los armónicos de orden superior corrigen detalles finos.
""")


# 4. Función para graficar la reconstrucción y calcular el error
def plot_reconstruccion(Na):
    ind = range(N - Na, N + Na + 1)
    er = 1 - np.sum(np.abs(cn[ind]) ** 2) / Px  # Error de reconstrucción

    # Señal reconstruida
    xe = phin[:, ind].dot(cn[ind])

    # Graficar la señal original y reconstruida
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(tv, x, color='r', label='$x(t)$ (Original)')
    ax.plot(tv, xe, color='b', label='$x_e(t)$ (Reconstruida)')
    ax.set_title(f'E_r = {100 * er:.2f}% (Error de reconstrucción)', fontsize=16)
    ax.set_xlabel('t [s]')
    ax.set_ylabel('x(t)')
    ax.grid(True)
    ax.legend()

    # Mostrar la gráfica en Streamlit
    st.pyplot(fig)

# Mostrar la señal original
st.header("Señal Original")
plot_original_signal()

# Control deslizante para elegir el número de armónicos utilizados en la reconstrucción en el tiempo
st.header("Señal con diferentes armónicos en el tiempo")
Na_time_val = st.slider('Número de armónicos en el tiempo (Na)', min_value=1, max_value=N, value=5)

# Mostrar la señal con los armónicos seleccionados en el tiempo
plot_harmonics_time(Na_time_val)

# Mostrar el espectro de Fourier
st.header("Espectro de Fourier")
plot_spectrum()

# Control deslizante para elegir el número de armónicos en la reconstrucción
st.header("Reconstrucción de la señal")
Na_val = st.slider('Número de armónicos en la reconstrucción (Na)', min_value=1, max_value=N, value=5)

# Mostrar la reconstrucción con el número de armónicos
plot_reconstruccion(Na_val)


Writing TFourier.py


3. Sea la señal portadora c(t) = Ac sin(2πFct), con Ac, Fc ∈ R, y la señal mensaje m(t) ∈ R. Encuentre el espectro en frecuencia de la señal modulada en amplitud (AM), y(t) = (1 + m(t)/Ac) c(t).
Luego, descargue desde YouTube 5 segundos de su canción favorita (capturando del segundo 20 al 25). Presente una simulación de modulación por amplitud AM (tomando como mensaje el fragmento de la canción escogida). Grafique las señales en tiempo y frecuencia (magnitud y fase) de la señal mensaje, portadora y modulada. Reproduzca los fragmentos de audio del mensaje, portadora y señal modulada.
Nota: se sugiere utilizar un canal de señal de audio para el desarrollo del ejercicio.
El usuario debe poder escoger el índice de modulación deseado.


# CÓDIGO DEL DASHBOARD

In [ ]:
!pip install streamlit -q #instalación de librerías
!pip install pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 1.9 MB/s eta 0:00:00


In [ ]:
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install soundfile #librerias descarga Youtube y manejo de audios en python

     - 2.7 MB 9.7 MB/s 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cached idna-3.8-py3-none-any.whl.metadata (9.9 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2024.8.30-py3-none-any.whl (167 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.3/157.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 48.0 MB/s eta 0:00:00
Using cached charse

In [ ]:
!pip install pydub #instalamos pydub para manipular el audio

In [ ]:
#Crear directorio pages
import os
try:
    os.mkdir('pages')
except FileExistsError:
    pass

In [ ]:
%%writefile pages/Modulacion.py

import streamlit as st #permite crear aplicaciones web interactivas directamente desde Python
import pandas as pd
import numpy as np
import os
import subprocess
import soundfile as sf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from pyngrok import ngrok
import yt_dlp as youtube_dl
import joblib

from scipy.fft import fft, fftfreq
from IPython.display import Audio
from pydub import AudioSegment
import io

# Intentar importar yt_dlp y manejar la falta de instalación
try:
    import yt_dlp
except ImportError:
    st.error("El módulo 'yt_dlp' no está instalado. Por favor, instálalo usando 'pip install yt-dlp'.")
    st.stop()  # Detener la ejecución si no se puede importar yt_dlp

# Descargar el audio de YouTube y convertir a WAV
def descargar_audio_youtube(url):
    ydl_opts = {
        'format': 'bestaudio[ext=m4a]',
        'outtmpl': 'audio.m4a'
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

    # Convertir a WAV
    audio = AudioSegment.from_file("audio.m4a", format="m4a")
    audio.export("audio.wav", format="wav")
    os.remove("audio.m4a")  # Limpiar el archivo m4a temporal

# Cargar el archivo de audio
def cargar_audio(filename):
    try:
        data, sample_rate = sf.read(filename)
        return sample_rate, data
    except FileNotFoundError:
        st.error(f"No se encontró el archivo {filename}. Asegúrate de que el archivo exista y esté en la ubicación correcta.")
        return None, None

# Convertir audio estéreo a mono
def convertir_a_mono(data):
    if len(data.shape) > 1:  # Si es estéreo
        return np.mean(data, axis=1)
    return data

# Graficar la señal en el tiempo
def graficar_senal_tiempo(t, signal, title, xlabel, ylabel):
    plt.figure(figsize=(10, 4))
    plt.plot(t, signal)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid()
    st.pyplot()

# Graficar la transformada de Fourier
def graficar_fft(t, signal, title):
    N = len(signal)
    T = t[1] - t[0]
    yf = np.fft.rfft(signal)
    xf = np.fft.fftfreq(N, T)[:N // 2]

    plt.figure(figsize=(10, 4))
    plt.plot(xf, np.abs(yf[0:N // 2]))
    plt.title(title)
    plt.xlabel('Frecuencia [Hz]')
    plt.ylabel('Magnitud')
    plt.grid()
    st.pyplot()

# Función principal de la app Streamlit
def main():
    st.title('Simulación de Modulación por Amplitud (AM)')

    st.write('La modulación AM es uno de los esquemas más simples de modulación de señales y se utiliza para transmitir información (mensaje) a través de ondas portadoras de radiofrecuencia. En este ejercicio, la señal portadora es:')
    st.latex(r'c(t) = A_c \sin(2\pi F_c t)')
    st.write('Donde $A_c$ es la amplitud de la portadora y $F_c$ su frecuencia. El mensaje $m(t)$, que es un fragmento de audio, es modulado sobre esta portadora, generando la señal modulada:')
    st.latex(r'y(t) = \left( 1 + \frac{m(t)}{A_c} \right) c(t)')
    st.write('Esta señal modulada se utiliza para transmitir la información del mensaje (audio) sobre frecuencias más altas, permitiendo que la señal sea enviada de manera eficiente a largas distancias.')

    url = st.text_input("Introduce el enlace del video de YouTube")

    if url:
        st.write("Descargando audio de YouTube...")
        descargar_audio_youtube(url)

        # Cargar el archivo de audio convertido
        sample_rate, data = cargar_audio("audio.wav")
        if data is None:
            return  # Salir si el archivo no se encuentra

        # Convertir a mono si es estéreo
        data_segment = convertir_a_mono(data)

        # Leer un fragmento de 5 segundos del archivo de audio
        ti = 20  # tiempo de inicio en segundos
        tf = 25   # tiempo final en segundos
        data_segment = data_segment[int(ti * sample_rate):int(tf * sample_rate)]

        # Tiempo de la señal
        t = np.arange(len(data_segment)) / sample_rate

        st.write("""El objetivo principal de esta simulación es visualizar
        y comprender cómo funciona la modulación en amplitud (AM), un método
        clásico en telecomunicaciones, al aplicar una señal de audio real como mensaje.
        A través de esta simulación, se exploran conceptos fundamentales en el procesamiento
        de señales, como la transformación de una señal baseband (mensaje) a una señal modulada,
        la representación en tiempo y frecuencia, así como la importancia de los
        índices de modulación en la calidad de la transmisión.""")

        st.write('El fragmento de audio capturado desde YouTube se toma como el mensaje $m(t)$. Es importante observar cómo se ve este audio en tiempo y en frecuencia.')

        # Mostrar la señal de audio original ###################################
        st.subheader('Señal de Audio Original')
        graficar_senal_tiempo(t, data_segment, 'Señal de Audio en el Tiempo', 'Tiempo [s]', 'Amplitud')

        # Reproducción de audio
        st.subheader('Reproducción de Audio')
        # Convertir el segmento de datos a WAV en un buffer de BytesIO
        wav_buffer = io.BytesIO()
        sf.write(wav_buffer, data_segment, sample_rate, format='wav')
        wav_buffer.seek(0)  # Reiniciar el puntero del buffer
        st.audio(wav_buffer, format='audio/wav')
        os.remove("audio.wav")

        st.write('La transformada de Fourier del mensaje $m(t)$ revela las frecuencias presentes en el fragmento de audio.')
        # Transformada de Fourier de la señal de audio
        st.subheader('Transformada de Fourier de la Señal de Audio')
        graficar_fft(t, data_segment, 'Espectro de la Señal de Audio')

        st.write('La señal portadora es una señal de alta frecuencia y se visualiza tanto en tiempo como en frecuencia. No lleva ninguna información en su forma original.')
        # Mostrar la señal portadora ###########################################
        st.subheader('Señal Portadora')
        # Parámetros de la portadora
        st.subheader('Parámetros de la Portadora')
        Fc = st.number_input('Frecuencia de la portadora (Hz)', value=15000)
        # Cálculo de la señal portadora y modulada
        st.write('Debido a que la señal portadora depende de el Indice de modulación,\neste aplica tanto para la portadora, cómo para la modulada')
        Im = st.slider('Índice de Modulación', 0.0, 5.0, 0.5)
        Ac = np.max(np.abs(data_segment)) / Im
        c = Ac * np.sin(2 * np.pi * Fc * t)
        y = (1 + data_segment / Ac) * c

        graficar_senal_tiempo(t, c, 'Señal Portadora en el Tiempo', 'Tiempo [s]', 'Amplitud') #Graficar portadora

        # Reproducción de portadora
        st.subheader('Reproducción de Portadora')

        archivo_portadora=io.BytesIO()
        sf.write(archivo_portadora, c, sample_rate, format='wav')
        archivo_portadora.seek(0)
        st.audio(archivo_portadora, format='audio/wav')

        # Transformada de Fourier de la señal portadora
        st.subheader('Transformada de Fourier de la Señal Portadora')
        graficar_fft(t, c, 'Espectro de la Señal Portadora')

        st.write("""La señal modulada $y(t)$ es la combinación de la portadora y el mensaje,
         y el espectro de esta señal muestra cómo la información del mensaje ha sido
         rasladada a una frecuencia más alta. Se representa la magnitud y fase de esta
         señal en el dominio de la frecuencia.""")
        # Mostrar la señal modulada ############################################
        st.subheader('Señal Modulada AM')

        graficar_senal_tiempo(t, y, 'Señal Moduladora AM en el Tiempo', 'Tiempo [s]', 'Amplitud')

        st.write('Al reproducir los fragmentos de audio del mensaje, la portadora y la señal modulada, se puede escuchar cómo el audio original $m(t)$ cambia cuando es modulado por la señal portadora.')
        st.write('La señal portadora, por sí sola, no tendrá información audible, ya que está en frecuencias demasiado altas para ser interpretadas como sonido.')
        st.write('La señal modulada en AM mantiene las características de la señal de audio original, pero ahora está "transportada" en una portadora de mayor frecuencia.')

        #Reproducción de señal modulada
        st.subheader('Reproducción de Señal modulada')

        archivo_modulada=io.BytesIO()
        sf.write(archivo_modulada, y, sample_rate, format='wav')
        archivo_modulada.seek(0)
        st.audio(archivo_modulada, format='audio/wav')

        # Transformada de Fourier de la señal modulada
        st.subheader('Transformada de Fourier de la Señal Modulada')
        graficar_fft(t, y, 'Espectro de la Señal Modulada')



if __name__ == "__main__":
    main()

Writing pages/Modulacion.py


4. Consulte en qué consiste la distorsión total de armónicos (Total Harmonic Distortion - THD) y el factor de potencia en un circuito eléctrico. ¿Cómo puede calcularse el THD desde la FFT? ¿Cómo puede calcularse la distorsión del factor de potencia con base al THD? Genere un ejemplo ilustrativo para el cálculo del THD y la distorsión del factor de potencia para un rectificador de onda completa con carga:
i) netamente resistiva.
ii) carga RC en serie.
Establezca las condiciones necesarias para las simulaciones. El usuario podrá escoger diferentes valores de R y C. Discuta los resultados obtenidos.

##Total Harmonic Distortion - THD
La THD mide la proporción de la potencia de los armónicos respecto a la potencia de la señal fundamental. La distorsión armónica ocurre cuando una señal se altera de manera que se añaden frecuencias adicionales (armónicos) a la señal original.

Se calcula mediante:

$\text{THD (%)} = \frac{\sqrt{P_2 + P_3 + P_4 + \cdots}}{P_1} \times 100 = \frac{\sqrt{V_2^2 + V_3^2 + \dots + V_n^2}}{V_1} \times 100$

Donde $P_1$ es la potencia de la frecuencia fundamental, y $P_2,P_3,...,P_n$ es la potencia para el armonico $n$ de la fundamental, y cada $V_i$ corresponde a la amplitud respectiva del armonico.


In [ ]:
%%writefile pages/Rectificador.py

#importamos las librerías necesarias
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt
import streamlit as st
import sympy as sym

# Configurar la página de Streamlit
st.set_page_config(page_title="Rectificador", layout="centered")

st.title("Cálculo del THD y la Distorsión del Factor de Potencia en un Rectificador de Onda Completa")
st.write("""
El objetivo de realizar un ejemplo ilustrativo para el cálculo del **THD (Total Harmonic Distortion o Distorsión Armónica Total)** y la **distorsión del factor de potencia** para un rectificador de onda completa con carga es comprender cómo estas características afectan la calidad y eficiencia de los sistemas de conversión de energía. Esto es especialmente importante en aplicaciones de electrónica de potencia, donde los rectificadores son comúnmente utilizados para convertir corriente alterna (AC) en corriente continua (DC).
""")

st.subheader("Distorsión Armónica Total (THD):")
st.write("""
El **THD** es una medida de la distorsión en una señal debido a la presencia de armónicos, los cuales son múltiplos de la frecuencia fundamental. Cuanto mayor sea el THD, más alejada está la señal de salida de una forma de onda ideal, lo que afecta el rendimiento de los equipos conectados.

En los sistemas de rectificación, la presencia de armónicos es común debido a la no linealidad del proceso de rectificación. Esta simulación permite al usuario ver cómo los diferentes valores de los componentes (resistencias y capacitancias) afectan el contenido armónico de la señal rectificada y, en consecuencia, el THD.
""")

st.subheader("Distorsión del Factor de Potencia (DFP):")
st.write("""
El **factor de potencia** mide cuán eficiente es el uso de la energía eléctrica en un sistema. Se calcula como la relación entre la potencia activa y la potencia aparente. La **distorsión del factor de potencia** indica cómo los armónicos impactan en la eficiencia de la transmisión de energía.

En cargas puramente resistivas, se espera que el factor de potencia esté más cerca de 1, lo que indica una conversión eficiente de la potencia. Sin embargo, en cargas inductivas o capacitivas, la distorsión del factor de potencia puede aumentar, lo que indica que hay más energía reactiva en el sistema que no contribuye a la potencia útil.
""")
st.markdown(r"""
Se calcula mediante:

$\text{THD (\%)} = \frac{\sqrt{P_2 + P_3 + P_4 + \cdots}}{P_1} \times 100 = \frac{\sqrt{V_2^2 + V_3^2 + \dots + V_n^2}}{V_1} \times 100$

Donde $P_1$ es la potencia de la frecuencia fundamental, y $P_2,P_3,...,P_n$ es la potencia para el armónico $n$ de la fundamental, y cada $V_i$ corresponde a la amplitud respectiva del armónico.
""")

# Parámetros de simulación ajustables
A = st.number_input('Amplitud de entrada (A)', value=120, step=10)
Fo = st.number_input('Frecuencia de entrada (Fo) en Hz', value=60, step=10)

# Parámetros de simulación
Fs = 30 * Fo  # Frecuencia de muestreo
To = 1 / Fo   # Periodo fundamental
Ts = 1 / Fs   # Periodo de muestreo
t = np.arange(0, 5 * To, Ts)  # Simulando 5 periodos
tau = None

# Selección del circuito: solo resistivo o resistivo-capacitivo
circuito = st.selectbox("Elija el tipo de circuito:", ["Solo Resistivo", "Resistivo-Capacitivo"])

# Parámetros del circuito
if circuito == "Solo Resistivo":
    st.write("""
    En este caso, se analizará el comportamiento de un rectificador de onda completa cuando la carga es solo una resistencia. Esto nos dará una referencia de cómo se comporta el sistema con la configuración más simple.
    """)
    R_v = st.number_input('Resistencia (R) en ohmios', value=1000, step=100)
    C_v = None  # No hay capacitancia en el circuito resistivo
    tau = R_v  # En el caso resistivo, el tiempo de respuesta depende solo de la resistencia
else:
    st.write("""
    Cuando la carga es un circuito **RC (resistencia y capacitancia en serie)**, el comportamiento cambia debido al almacenamiento de energía en el condensador. El condensador suaviza la señal rectificada, lo que modifica tanto el contenido armónico como el factor de potencia.

    En esta simulación, se permite al usuario variar los valores de **R** y **C** para observar cómo estos afectan tanto el **THD** como la **distorsión del factor de potencia**.
    """)
    R_v = st.number_input('Resistencia (R) en ohmios', value=1000, step=100)
    C_v = st.number_input('Capacitancia (C) en faradios', value=10e-6, format="%.2e",step=10e-6)
    tau = R_v * C_v  # En el circuito RC, el tiempo de respuesta depende de R y C ##

# Función de entrada: señal senoidal rectificada completa
in_o = A * np.sin(2 * np.pi * Fo * t) #Señal sinosoidal de alimentación
in_ = abs(in_o) # Entrada rectificada completa

# Cálculo de la respuesta del circuito
if C_v is None:  # Solo resistivo
    st.write("""
    En una carga resistiva, el contenido armónico es más elevado, ya que la resistencia no tiene la capacidad de filtrar los picos de corriente. El **THD** será generalmente más alto, pero el **factor de potencia** tiende a ser cercano a 1, ya que no hay energía reactiva involucrada.
    """)
    out = in_   # Respuesta del circuito resistivo
else:
  # Función de transferencia para el circuito RC

  #out = sig.lfilter(num, den, in_)  # Salida del circuito RC usando scipy.signal.lfilter
  st.write("""
  Al introducir una capacitancia en la carga, la señal de salida será más suave debido al filtrado que proporciona el condensador. Esto reduce el contenido armónico, disminuyendo el **THD**. Sin embargo, la presencia de la capacitancia también introduce energía reactiva, lo que puede reducir el **factor de potencia**.
  """)
  s = sym.symbols('s', complex=True)
  to, R, L, C = sym.symbols('t R L C', positive=True)
  X = sym.Function('X')(s)
  Y = 1/(R*C*s + 1) * X

  to=t

  Y_RC = Y.subs(R, R_v).subs(C, C_v)

  num = np.array([1])  # Numerador
  den = np.array([R_v * C_v, 1])  # Denominador
  G_n = sig.TransferFunction(num, den)

  out = G_n.output(in_, T=to)[1]

# Mostrar gráficos en Streamlit
st.write(f"### Señal de Alimentación ({circuito})")
st.line_chart(in_o)

st.write(f"### Señal Rectificada y Salida del Circuito ({circuito})")
fig, ax = plt.subplots()
plt.plot(t, in_, label='Entrada (In)')
plt.plot(t, out, label='Salida (Out)')
plt.xlabel('Tiempo [s]')
plt.ylabel('Voltaje [V]')
plt.legend()
#plt.show()
st.pyplot(fig)

# Cálculo del espectro de frecuencias (FFT)
vfre = np.fft.rfftfreq(len(out), 1 / Fs)
Xf = np.fft.rfft(out)

st.write("### Espectro de Frecuencias de la Salida (FFT)")
fig, ax = plt.subplots()
ax.stem(vfre, np.abs(Xf) / len(out), use_line_collection=True)
ax.set_xlabel('Frecuencia [Hz]')
ax.set_ylabel('|X(f)|')
st.pyplot(fig)

# Cálculo del THD
def calculate_thd(signal):
    N = len(signal)
    Y = np.fft.rfft(signal)
    Y = 2.0 / N * np.abs(Y[:N // 2])
    fundamental = Y[1]  # Fundamental está en la posición 1 (no 0 debido al DC)
    harmonics = np.sqrt(np.sum(Y[2:]**2))  # Suma de las componentes armónicas
    THD = harmonics / fundamental
    return THD

THD = calculate_thd(out)
st.write(f"### THD (Distorsión Armónica Total) para el circuito {circuito}: {THD:.2%}")

# Cálculo del factor de potencia (FP)
def calculate_power_factor(THD):
  return np.sqrt(1/(1 + THD**2))

FP = calculate_power_factor(THD)
st.write(f"### Factor de Potencia para el circuito {circuito}: {FP:.2f}")


Writing pages/Rectificador.py


In [ ]:
#cambiar su token
usuario="Cristian"
if usuario=="Yuliana":
  token = '2lLjHoDPtNqGFhA3XCMXaV7TSzC_2WMetgXt54NgDAv4qBuZE' #colocar aquí su token personal después de crear su cuenta con correo UNAL en Ngrok
elif usuario=="Juan":
  token = '2lLkHdYMBU6KwzPcVAaxaNJyYYj_5y1Y8DSwG18FHt3fkdFzX'
elif usuario=='Cristian':
  token = '2lLjBy3NA0GhfCDLgRI47jQwMpL_4FDAUMmnURPshTKsBFXY8'

In [ ]:
from pyngrok import ngrok

# Set authentication token (unique per user)
ngrok.set_auth_token(token)

# Start Streamlit server on a specific port
!nohup streamlit run TFourier.py --server.port 8600 & #cambiar el puerto 8499 pueden poner cualquier número de 4 digitos

# Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='8600', proto='http', bind_tls=True) #acá tambien lo tienen que cambiar

# Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://4225-34-23-234-53.ngrok-free.app


In [ ]:
#exit("Stopping the execution")